In [1]:
import logging
import grpc
import pandas as np
import numpy as np
import time

from proto import contest_pb2
from proto import contest_pb2_grpc
# from proto import contest_pb2
from proto import question_pb2
# from proto import question_pb2


ModuleNotFoundError: No module named 'contest_pb2'

In [13]:
contest_channel=grpc.insecure_channel('47.103.23.116: 56702')
question_channel=grpc.insecure_channel('47.103.23.116: 56701')

contest_stub=contest_pb2_grpc.ContestStub(contest_channel)
question_stub=question_pb2_grpc.QuestionStub(question_channel)

login_response=contest_stub.login(contest_pb2.LoginRequest(user_id=88,user_pin='dDTSvdwk'))

_Rendezvous: <_Rendezvous of RPC that terminated with:
	status = StatusCode.UNKNOWN
	details = "Stream removed"
	debug_error_string = "{"created":"@1608123801.303000000","description":"Error received from peer ipv4:47.103.23.116:56702","file":"src/core/lib/surface/call.cc","file_line":1055,"grpc_message":"Stream removed","grpc_status":2}"
>

In [16]:
sequence = 0
last_sequence = 0
while sequence < 1000:
    question_response=question_stub.get_question(question_pb2.QuestionRequest(user_id=88))
    sequence = question_response.sequence
    if last_sequence < sequence:
        # Run our strategy here, resulting in a POS array with length 351

        dailystk = [x.values for x in question_response.dailystk]
        ds = np.asarray(dailystk)

        # Here is our target array
        # pos = np.sign(np.random.rand(351) - 0.5) * np.floor(question_response.capital / 351 / ds[:,5])
        pos = pos * -1
        # summit answer
        submit_response = contest_stub.submit_answer(contest_pb2.AnswerRequest(user_id=88,user_pin='dDTSvdwk',session_key=login_response.session_key,sequence=sequence,positions=pos))
        print(submit_response)
        time.sleep(1)
        break

accepted: true



In [4]:
question_response=question_stub.get_question(question_pb2.QuestionRequest(user_id=88))

In [5]:
question_response.capital

483828965.01277775